In [1]:
%matplotlib notebook

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import scipy.stats as sts
import seaborn as sns
import time
import seaborn

# Import API key
from api_keys import g_key
from api_keys import api_key
from api_keys import weather_api_key
#gmaps.configure(api_keys= g_key)


In [3]:
city_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#data =  city_url + "lompoc" + "&APPID=" + api_key
#response = requests.get(city_url)
#data = response.json()
#pprint(data)

#import openweathermapy as gmaps


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
#Loading in WeatherPY CSV

input_file = "cities.csv"


csv=pd.read_csv(input_file)

csv.head()


,city_id,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hithadhoo,-0.6000,73.0833,81.34,71,92,7.78,MV,1627858049
1,1,Yellowknife,62.4560,-114.3525,87.33,28,20,3.44,CA,1627858059
2,2,Saint-Joseph,-21.3667,55.6167,64.78,75,37,8.30,RE,1627858292
3,3,Cidreira,-30.1811,-50.2056,56.86,93,90,8.03,BR,1627858086
4,4,Talara,-4.5772,-81.2719,65.91,84,1,21.12,PE,1627858137


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [6]:


#Creating necesarry variables for Location and Humidity
locations=csv[["Lat", "Lng"]]
Humidity= csv["Humidity"]

#Define the heatmap parameters
fig=gmaps.figure()

#Create heat layer
heat_layer=gmaps.heatmap_layer(locations, weights=Humidity, dissipating=False, 
                              point_radius=3)

#Add Layer
fig.add_layer(heat_layer)

#Display the map
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#Personal preference
    #80> but not higher 90 
    #50< Humidity
    #Lat > 20

temp_df=csv[(csv["Max Temp"]>=80)&(csv["Max Temp"]<90)&(csv["Humidity"]<50)& (csv["Lat"]>20)]
temp_df

,city_id,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,Yellowknife,62.4560,-114.3525,87.33,28,20,3.44,CA,1627858059
13,13,Norman Wells,65.2820,-126.8329,87.84,23,40,4.61,CA,1627858085
45,45,La Ronge,55.1001,-105.2842,86.25,25,75,8.05,CA,1627858302
83,83,Haines Junction,60.7522,-137.5108,80.55,32,16,6.91,CA,1627858310
100,100,Esna,25.2934,32.5540,87.82,14,42,6.69,EG,1627858315
112,112,Thompson,55.7435,-97.8558,84.36,30,20,4.61,CA,1627858317
130,130,Koilada,37.4095,23.1147,84.58,43,0,6.26,GR,1627858322
230,230,Aklavik,68.2191,-135.0107,84.92,39,99,15.50,CA,1627858092
243,243,Sakakah,29.9697,40.2064,87.37,14,0,10.20,SA,1627858352
279,279,Ardakān,32.3100,54.0175,86.38,14,37,6.49,IR,1627858360


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
#create df of hotel name, along with city, count, geocod(lat,long)
hotel_df=temp_df[["City","Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Names"]= ""
hotel_df

,City,Country,Lat,Lng,Hotel Names
1,Yellowknife,CA,62.4560,-114.3525,
13,Norman Wells,CA,65.2820,-126.8329,
45,La Ronge,CA,55.1001,-105.2842,
83,Haines Junction,CA,60.7522,-137.5108,
100,Esna,EG,25.2934,32.5540,
112,Thompson,CA,55.7435,-97.8558,
130,Koilada,GR,37.4095,23.1147,
230,Aklavik,CA,68.2191,-135.0107,
243,Sakakah,SA,29.9697,40.2064,
279,Ardakān,IR,32.3100,54.0175,


In [9]:
#Define variables
radius=8000
gtype="lodging"

#Making query calls for top Hotel

base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params={"radius": radius, "types": gtype,"key": g_key}

In [10]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    #Save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Could not find result")
        print("-------------------------")
        #delay the code by one second to avoid an error
        time.sleep(1)
    print("--------------------Ending Search")

Retrieving Results for Index 1: Yellowknife.
Closest hotel in Yellowknife is The Explorer Hotel.
--------------------Ending Search
Retrieving Results for Index 13: Norman Wells.
Closest hotel in Norman Wells is Heritage Hotel.
--------------------Ending Search
Retrieving Results for Index 45: La Ronge.
Closest hotel in La Ronge is Northland Motor Hotel.
--------------------Ending Search
Retrieving Results for Index 83: Haines Junction.
Closest hotel in Haines Junction is Parkside Inn.
--------------------Ending Search
Retrieving Results for Index 100: Esna.
Closest hotel in Esna is Orient Star Dahabyia Isna 05 Nights Each Sunday.
--------------------Ending Search
Retrieving Results for Index 112: Thompson.
Closest hotel in Thompson is Burntwood Hotel.
--------------------Ending Search
Retrieving Results for Index 130: Koilada.
Closest hotel in Koilada is Costa Bianca.
--------------------Ending Search
Retrieving Results for Index 230: Aklavik.
Closest hotel in Aklavik is Aklavik Inn.
-

In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
locations

,Lat,Lng
1,62.4560,-114.3525
13,65.2820,-126.8329
45,55.1001,-105.2842
83,60.7522,-137.5108
100,25.2934,32.5540
112,55.7435,-97.8558
130,37.4095,23.1147
230,68.2191,-135.0107
243,29.9697,40.2064
279,32.3100,54.0175


In [12]:
hotel_df=hotel_df[hotel_df["Hotel Names"]!= ""]
hotel_df

,City,Country,Lat,Lng,Hotel Names,Hotel Name


In [13]:
# Plot markers on top of the heatmap
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))